# Day 15

link: https://adventofcode.com/2022/day/15

ก่อนอื่น เรา parse input เก็บเฉพาะ coordinate ของ sensor และ beacon

In [1]:
input = IO.foreach('data/15.txt').to_a.map(&:strip)

sensors = input.map{|line|
  words = line.split(/[\s\,=]/)
  [words[3].to_i, words[6].to_i, words[12].to_i, words[15].to_i]
}
nil

## Part 1

เราสามารถหา range ของ sensor ได้ จาก manhattan distance ระหว่างคู่ sensor กับ beacon
สมมติให้ range นี้ มีค่า `d`

ข้อนี้เราสนใจเฉพาะ row ที่ 2000000 เราก็ลากเส้นตรงจาก sensor ในแนวดิ่งไปที่ row ที่ 2000000 ให้เส้นนี้ยาว `dy`

- ถ้า `dy > d` แปลว่า row นี้อยู่นอกระยะของ sensor เราก็ ignore ได้
- ถ้า `dy <= d` ก็จะยังเหลือ range ของ sensor อยู่อีก `dx = d - dy` ซึ่งเราสามารถบิดไปทางซ้ายหรือขวาก็ได้ ดังนั้น sensor ตัวนี้จะครอบคลุมตำแหน่ง `x - dx` ไปจนถึง `x + dx`

เราก็หา range ของ sensor ทุกตัว เอามา merge กัน แล้วนับ ก็จะได้จำนวนตำแหน่งที่ไม่สามารถมี beacon อยู่ได้ _ยกเว้น_ ตำแหน่งที่มี beacon อยู่จริงๆ ต้องเอาไปลบออก

In [2]:
def try_merge(occupied, i)
  while i >= 0 && occupied[i] && occupied[i+1] && occupied[i][1] >= occupied[i+1][0] - 1
    r1, r2 = occupied.delete_at(i+1)
    occupied[i][1] = r2 if r2 > occupied[i][1]
  end
end

def find_occupied_range(sensors, y, min_x, max_x)
  occupied = []
  sensors.each{|sx, sy, bx, by|
    d = (bx-sx).abs + (by-sy).abs
    dy = (sy - y).abs
    dx = d - dy
    if dx >= 0
      x1 = sx - dx
      x2 = sx + dx

      x1 = min_x if min_x && x1 < min_x
      x2 = max_x if max_x && x2 > max_x

      i = 0
      i += 1 while occupied[i] && occupied[i][0] <= x1

      occupied.insert(i, [x1, x2])
      
      try_merge(occupied, i - 1)
      try_merge(occupied, i)
      
    end
  }
  occupied
end

target_row = 2000000
occupied = find_occupied_range(sensors, target_row, nil, nil)
occupied_size = occupied.map{|x1, x2| x2-x1 + 1}.sum

existing_beacons = sensors.select{|sx, sy, bx, by| by == target_row}.map{|sx, sy, bx, by| bx}.uniq

overcount = existing_beacons.count{|bx|
  occupied.any?{|x1, x2| x1 <= bx && bx <= x2}
}

puts occupied_size - overcount

5838453


## Part 2

part หลังเราไม่ได้ทำอะไรซับซ้อนมาก แค่เพิ่ม constraint ว่า hidden beacon นี้ต้องอยู่ใน range (0, 0) ถึง (4000000, 4000000)
เราก็ใช้ method เดิมวนไป 4000000 รอบ หา row ที่มี disjointed range ซึ่งโจทย์ยืนยันว่าจะมีอยู่แค่ row เดียว กินเวลานิดหน่อยแต่ก็ไม่นานจนเกินไป

In [3]:
m = 4000000
0.upto(m){|y|
  occupied = find_occupied_range(sensors, y, 0, m)
  if occupied.size == 2 # would miss if the beacon is at x = 0 or x = m
    puts y + m * (occupied[0][1] + 1)
    break
  end
}

12413999391794
